In [11]:
import numpy as np
import numba
from numba import typed
#import matplotlib.pyplot as plt
#from matplotlib.animation import FuncAnimation
#from IPython.display import HTML

import random
from PIL import Image, ImageDraw
import math

import tqdm
import time

In [12]:
n = 40 # rozmiar siatki
m = n # tego nie było, ale w sumie czemu nie - siatka nie musi być kwadratowa
J = 1
beta = 1
B = 1
liczba_krokow = 100 # liczba kroków; makrokroków 1 * liczba spinów
gestosc_spinow = 0.5 # początkowa gęstość spinów

# opcjonalne:
prefix = 'ising' # nazwę pliku z obrazkami (prefix, do którego program dodaje mumer kroku; jeżeli parametr
#  nie zostanie podany, obrazki się nie generują)
animacja = 'animacja' # nazwę pliku z animacją (jeżeli parametr nie zostanie podany, animacja się nie generuje)
plik_magnetyzacja = 'magnetyzacja' # nazwę pliku z magnetyzacją (jeżeli parametr nie zostanie podany, magnetyzacja się nie zapisuje)

obrazki = []
obrazkiNumba = []

magnetyzacje = []
magnetyzacjeN = []

# bez Numby

In [13]:
def nowa_siatka(n, m, gestosc_spinow):
    siatka = np.zeros((n, m))

    for i in range(n):
        for j in range(m):
            spin = random.random() #np.random.rand(1)
            if spin < gestosc_spinow :
                spin = 1
            else:
                spin = -1
            siatka [i][j] = spin
    return siatka

def liczenie_energii(n, m, J, B, magnetyzacja, numba_siatka):
    energia1 = 0 # czyli tylko suma iloczynów spinów
    for i in range(n-1):
        for j in range(m-1):
            energia1 = energia1 + numba_siatka[i][j] *(numba_siatka[i+1][j] + numba_siatka[i][j+1] )
    for i in range(n-1):
        energia1 = energia1 + numba_siatka[i][j] *(numba_siatka[i+1][j] + numba_siatka[i][1] )
    for j in range(m-1):
        energia1 = energia1 + numba_siatka[i][j] *(numba_siatka[1][j] + numba_siatka[i][j+1] )
    energia1 = energia1 + numba_siatka[n-1][m-1] *(numba_siatka[1][m-1] + numba_siatka[n-1][1] )
    energia =  - J*energia1 - B*magnetyzacja
    #   print('Energia: ', energia)
    return energia

def mikrokrok(n, m, beta, energia_stara, J, B, magnetyzacja, numba_siatka):
    i = random.randrange(n)
    j = random.randrange(m)
        # print(i, j)
    numba_siatka[i][j] = - numba_siatka[i][j] 
    energia_nowa = liczenie_energii(n, m, J, B, magnetyzacja, numba_siatka) # liczymy jeszcze raz energię

    if (energia_nowa < energia_stara):
        pass # akceptujemy nowy spin
    else:
        p = random.random() #np.random.rand(1)
        P_delta_E = math.exp(- beta * (energia_nowa - energia_stara) )
        if(p < P_delta_E):
            pass # akceptujemy nowy spin
        else:
            numba_siatka[i][j] = - numba_siatka[i][j] #powrót do poprzedniego stanu
    return energia_nowa

def licz_magnetyzacje(n, m, nr, obrazki, numba_siatka): # i przy okazji rysuj obrazek
    magnetyzacja = 0
    img = Image.new('RGB', (10*n + 4, 10*m + 4), (255, 255, 255)) # rozmiar obrazka zmienia się w zależności od rozmiarów siatki
    klatka = ImageDraw.Draw(img)
    wys = 10
    for i in range(n):
        for j in range(m):
            spin = numba_siatka[i][j]
            if spin == 1 :
                klatka.multiline_text((4 + i*wys, j*wys), "+", fill=(0, 0, 0))
            else:
                klatka.multiline_text((4+i*wys, j*wys), "-", fill=(0, 0, 0))
                    
            magnetyzacja = magnetyzacja + spin
    magnetyzacja_cała = magnetyzacja / (n*m)
        #print('Magnetyzacja: ', magnetyzacja_cała)
#        img.show()
    if (prefix != 'brak'):
        img.save(prefix + str(nr) +'.png') # dodać warunek na brak nazwy
    obrazki.append(img)
    return magnetyzacja_cała

def makrokroki(n, m, J, B, beta, liczba_krokow, obrazki, magnetyzacje, numba_siatka):
    for i in tqdm.tqdm(range(liczba_krokow)):
        magnetyzacja = licz_magnetyzacje(n, m, i, obrazki, numba_siatka)
            
        magnetyzacje.append(magnetyzacja)
        #    print('Magnetyzacja: ', magnetyzacja)
                #zapisanie magnetyzacji do pliku
        magnetyzacja_suma = magnetyzacja * (n*m)
        stara_energia = liczenie_energii(n, m, J, B, magnetyzacja_suma, numba_siatka) 

        for k in range(n*m):
            energia = mikrokrok(n, m, beta, stara_energia, J, B, magnetyzacja_suma, numba_siatka) 
            stara_energia = energia

In [14]:
siatka1 = nowa_siatka(n, m, gestosc_spinow)
makrokroki(n, m, J, B, beta, liczba_krokow, obrazki, magnetyzacje, siatka1)

100%|██████████| 100/100 [03:10<00:00,  1.91s/it]


In [15]:
if animacja != 'brak':      
    obrazki[0].save(animacja + '.gif',
               save_all=True, append_images=obrazki[1:], duration=100, loop=1)
               # loop odpowiada za to, czy filmik zapętla się w nieskończoność, czy nie

# Numba

In [16]:
@numba.njit
def nowa_siatkaN(n, m, gestosc_spinow):
    siatka = np.zeros((n, m))

    for i in range(n):
        for j in range(m):
            spin = random.random() #np.random.rand(1)
            if spin < gestosc_spinow :
                spin = 1
            else:
                spin = -1
            siatka [i][j] = spin
    return siatka

In [17]:
@numba.njit
def liczenie_energiiN(n, m, J, B, magnetyzacja, numba_siatka):
    energia1 = 0 # czyli tylko suma iloczynów spinów
    for i in range(n-1):
        for j in range(m-1):
            energia1 = energia1 + numba_siatka[i][j] *(numba_siatka[i+1][j] + numba_siatka[i][j+1] )
    for i in range(n-1):
        energia1 = energia1 + numba_siatka[i][j] *(numba_siatka[i+1][j] + numba_siatka[i][1] )
    for j in range(m-1):
        energia1 = energia1 + numba_siatka[i][j] *(numba_siatka[1][j] + numba_siatka[i][j+1] )
    energia1 = energia1 + numba_siatka[n-1][m-1] *(numba_siatka[1][m-1] + numba_siatka[n-1][1] )
    energia =  - J*energia1 - B*magnetyzacja
    #   print('Energia: ', energia)
    return energia

In [18]:
@numba.njit
def mikrokrokN(n, m, beta, energia_stara, J, B, magnetyzacja, numba_siatka):
    i = random.randrange(n)
    j = random.randrange(m)
        # print(i, j)
    numba_siatka[i][j] = - numba_siatka[i][j] 
    energia_nowa = liczenie_energiiN(n, m, J, B, magnetyzacja, numba_siatka) # liczymy jeszcze raz energię

    if (energia_nowa < energia_stara):
        pass # akceptujemy nowy spin
    else:
        p = random.random() #np.random.rand(1)
        P_delta_E = math.exp(- beta * (energia_nowa - energia_stara) )
        if(p < P_delta_E):
            pass # akceptujemy nowy spin
        else:
            numba_siatka[i][j] = - numba_siatka[i][j] #powrót do poprzedniego stanu
    return energia_nowa

In [20]:
#@numba.njit
def licz_magnetyzacjeN(n, m, nr, obrazki, numba_siatka): # i przy okazji rysuj obrazek
    magnetyzacja = 0
    img = Image.new('RGB', (10*n + 4, 10*m + 4), (255, 255, 255)) # rozmiar obrazka zmienia się w zależności od rozmiarów siatki
    klatka = ImageDraw.Draw(img)
    wys = 10
    for i in range(n):
        for j in range(m):
            spin = numba_siatka[i][j]
            if spin == 1 :
                klatka.multiline_text((4 + i*wys, j*wys), "+", fill=(0, 0, 0))
            else:
                klatka.multiline_text((4+i*wys, j*wys), "-", fill=(0, 0, 0))
                    
            magnetyzacja = magnetyzacja + spin
    magnetyzacja_cała = magnetyzacja / (n*m)
        #print('Magnetyzacja: ', magnetyzacja_cała)
#        img.show()
    if (prefix != 'brak'):
        img.save(prefix + 'Numba' + str(nr) +'.png') # dodać warunek na brak nazwy
    obrazkiNumba.append(img)
    return magnetyzacja_cała

In [21]:
#@numba.njit
def makrokrokiN(n, m, J, B, beta, liczba_krokow, obrazki, magnetyzacje, numba_siatka):
    for i in tqdm.tqdm(range(liczba_krokow)):
        magnetyzacja = licz_magnetyzacjeN(n, m, i, obrazki, numba_siatka)
            
        magnetyzacjeN.append(magnetyzacja)
        #    print('Magnetyzacja: ', magnetyzacja)
                #zapisanie magnetyzacji do pliku
        magnetyzacja_suma = magnetyzacja * (n*m)
        stara_energia = liczenie_energiiN(n, m, J, B, magnetyzacja_suma, numba_siatka) 

        for k in range(n*m):
            energia = mikrokrokN(n, m, beta, stara_energia, J, B, magnetyzacja_suma, numba_siatka) 
            stara_energia = energia

In [22]:
numba_siatka1 = nowa_siatkaN(n, m, gestosc_spinow)
makrokrokiN(n, m, J, B, beta, liczba_krokow, obrazkiNumba, magnetyzacje, numba_siatka1)

100%|██████████| 100/100 [00:03<00:00, 32.15it/s]


In [23]:
if animacja != 'brak':      
    obrazkiNumba[0].save(animacja + 'Numba' + '.gif',
               save_all=True, append_images=obrazkiNumba[1:], duration=100, loop=1)
               # loop odpowiada za to, czy filmik zapętla się w nieskończoność, czy nie